# Name: Yu Fo Hon

# Student ID: 20201702

In [126]:
def print_color(text, color):
    text=str(text)
    color_code = {
        'black': '\u001b[30m',
        'red': '\u001b[31m',
        'green': '\u001b[32m',
        'yellow': '\u001b[33m',
        'blue': '\u001b[34m',
        'magenta': '\u001b[35m',
        'cyan': '\u001b[36m',
        'white': '\u001b[37m',
        'reset': '\u001b[0m'
    }

    color_name = color.lower()
    if color_name in color_code:
        colored_text = color_code[color_name] + text + color_code['reset']
        print(colored_text)
    else:
        print(text)

In [127]:
def print_color_multiple(*args, color):
    color_code = {
        'black': '\u001b[30m',
        'red': '\u001b[31m',
        'green': '\u001b[32m',
        'yellow': '\u001b[33m',
        'blue': '\u001b[34m',
        'magenta': '\u001b[35m',
        'cyan': '\u001b[36m',
        'white': '\u001b[37m',
        'reset': '\u001b[0m'
    }

    color_name = color.lower()
    if color_name in color_code:
        colored_text = color_code[color_name] + " ".join(str(arg) for arg in args) + color_code['reset']
        print(colored_text)
    else:
        print(*args)

In [128]:
def get_value(dictionary, key, default_value):
    try:
        return dictionary[key]
    except KeyError as e:
        print_color_multiple(f"Error: {e} is missing in the invoice data, so system will use default value: ",default_value, color='red')
        return default_value

In [129]:
class InvoiceVerifier:
    def __init__(self, invoice):
        self.invoice = invoice

    def calculate_check_digit(self, staff_number, modulus_char, order_number):
        # Calculation of check digit
        multipliers = [int(digit) for digit in staff_number]
        sum_of_products = sum(int(order_number[i]) * multipliers[i] for i in range(len(order_number)))
        # Find the check digit such that sum_of_products + check_digit is divisible by check_digit_modulus
        check_digit = 0
        while (sum_of_products + check_digit) % self.get_check_digit_modulus(modulus_char) != 0:
            check_digit += 1
        return check_digit
    
    def calculate_hash_total(self, item_numbers):
        # Calculate the sum of the last 2 digits of each item number
        hash_total = sum(int(str(item)[-2:]) for item in item_numbers)
        return hash_total

    def calculate_mall_dollars(self, sub_total):
        return sub_total * 0.1

    def calculate_discounts(self, sub_total,VIPXX_str):
        vip_discount_str = VIPXX_str
        percentage_str = vip_discount_str[3:5]
        vipday_percentage = int(percentage_str) / 100
        vipday_discount = sub_total * (1-vipday_percentage)
        return round(vipday_discount,2)

    def calculate_delivery_fee(self, sub_total):
        return 0 if sub_total >= 500 else 50
    
    def calculate_subtotal(self, items):
        calculated_sub_total = 0.0
        for item in items:
            item_data = item.split(', ')
            price = float(item_data[5])
            calculated_sub_total += price
        return calculated_sub_total
    
    @staticmethod
    def get_check_digit_modulus(modulus_char):
        # Mapping of alphabets to modulus values
        modulus_values = {'A': 9, 'B': 8}
        return modulus_values[modulus_char]

    @staticmethod
    def validate_invoice_format(order_number):
        # Check if the length of order number is correct
        is_correct_length = len(order_number) - 2 == 16
        print_color("Order number length:", 'black')
        print('✅' if is_correct_length else '❌')
        
        # Check if the modulus character is correct (A or B)
        has_correct_modulus_char = order_number[7] in "AB"
        print_color("Modulus character:", 'black')
        print('✅' if has_correct_modulus_char else '❌')

        # Check if the specified parts of the order number are digits
        has_digits = order_number[1:7].isdigit() and order_number[8:14].isdigit()
        print_color("Digits in specified parts:", 'black')
        print('✅' if has_digits else '❌')

        # Check if item count format is a digit
        has_correct_item_count_format = order_number[14].isdigit()
        print_color("Item count format:", 'black')
        print('✅' if has_correct_item_count_format else '❌')

        # Check if check digit format is a digit
        has_correct_check_digit_format = order_number[-2].isdigit()
        print_color("Check digit format:", 'black')
        print('✅' if has_correct_check_digit_format else '❌')

        # Check if parentheses are correct
        has_correct_parentheses = order_number[-3] == '(' and order_number[-1] == ')'
        print_color("Parentheses format:", 'black')
        print('✅' if has_correct_parentheses else '❌')

        # Check if first character is an uppercase letter A-Z
        is_first_char_uppercase_letter = 'A' <= order_number[0] <= 'Z'
        print_color("First character:", 'black')
        print('✅' if is_first_char_uppercase_letter else '❌')

        # Combine all checks to determine overall validity
        is_valid = (
            is_correct_length and
            has_correct_modulus_char and
            has_digits and
            has_correct_item_count_format and
            has_correct_check_digit_format and
            has_correct_parentheses and
            is_first_char_uppercase_letter
        )

        return is_valid
    
    @classmethod
    def from_file(cls, file_path):
        with open(file_path, 'r') as file:
            invoice_data_array = []
            current_invoice = {'items':[]}

            for line in file:
                line = line.strip()
                if 'Test Data' in line:
                    continue
                if ',' in line:
                    if 'Item' in line:
                        current_invoice['items'].append(line)
                    elif 'VIP' in line and line[4:5].isdigit():
                        key, value = map(str.strip, line.split(',', 1))
                        current_invoice['VIPXX_str']=key
                        current_invoice['VIPXX_discount']=value
                    else:
                        key, value = map(str.strip, line.split(',', 1))
                        current_invoice[key] = value
                else:
                    invoice_data_array.append(current_invoice)
                    current_invoice = {'items':[]}
                    
            # Append the last invoice if it exists 
            if current_invoice:
                invoice_data_array.append(current_invoice)  
                
        return invoice_data_array
    
    def verify_invoice(self):
        order_number = self.invoice['order number']
        
        if not self.validate_invoice_format(order_number):
            print_color("❌ Invalid invoice format.","red")
            
        staff_number = order_number[1:7]
        modulus_char = order_number[7]
        order_number_sequence = order_number[8:14]
        items_count = int(order_number[14])
        provided_check_digit = int(order_number[-2])

        calculated_check_digit = self.calculate_check_digit(staff_number, modulus_char, order_number_sequence)

        if provided_check_digit != calculated_check_digit:
            print('❌Invalid check digit. check_digit should be: ', end="")
            print_color(calculated_check_digit,'red')
        else:
            print_color_multiple("✅Correct check_digit: ",calculated_check_digit,color="green")
            
        vip_discount = float(get_value(self.invoice, 'VIP discount', 0.0))
        vipday_discount = float(get_value(self.invoice, 'VIPXX_discount', 0.0))
        sub_total = float(get_value(self.invoice, 'sub_total', 0.0))
        total_items = int(get_value(self.invoice, 'number of items', 0))
        items = get_value(self.invoice, 'items', [])
        VIPXX_str = get_value(self.invoice, 'VIPXX_str', "")
        mall_dollars = float(get_value(self.invoice, 'Mall dollars', 0.0))
        # Calculate subtotal based on item prices and quantities
        calculated_sub_total = self.calculate_subtotal(items)
        
        # Check if the calculated subtotal matches the provided subtotal
        if abs(calculated_sub_total - sub_total) > 1e-9:
            print_color_multiple('❌Invalid subtotal,subtotal should be :',str(calculated_sub_total),",but input is: " + str(sub_total),color='red')
        else:
            print_color_multiple("✅Correct subtotal: ",calculated_sub_total,color="green")
            
        # Calculate the VIP and VIPDAY discounts
        if(vip_discount>-2 or vip_discount<-20):
            print_color_multiple('❌Invalid VIP_discount,VIP_discount should be range from 2 to 20 but input vip_discount is',vip_discount,color='red')
        else:
            print_color_multiple("✅Correct vip discount: ",vip_discount,color="green")
        
        calculated_sub_total += vip_discount
        
        vipday_discount = self.calculate_discounts(calculated_sub_total,VIPXX_str)
        if(-vipday_discount != float(self.invoice['VIPXX_discount'])):
            print_color_multiple('❌Invalid VIPDAY_discount,VIPDAY_discount should be ',str(-vipday_discount) +",but input is: " + self.invoice['VIPXX_discount'],color='red')
        else:
            print_color_multiple("✅Correct " + self.invoice['VIPXX_str'] ,-vipday_discount,color="green")
            
        # Calculate the delivery fee
        delivery_fee = self.calculate_delivery_fee(calculated_sub_total)
        if(delivery_fee != float(self.invoice['Delivery Fee'])):
            print_color_multiple('❌Invalid delivery_fee,delivery_fee should be ',str(delivery_fee) +",but input is: " + self.invoice['Delivery Fee'],color='red')
        else:
            print_color_multiple("✅Correct delivery fee: ",delivery_fee,color="green")
            
        # Calculate the total amount
        calculated_total = calculated_sub_total  - vipday_discount - delivery_fee
        # Check if the calculated total matches the provided total
        if abs(calculated_total - float(self.invoice['Total'])) > 1e-9:
            print_color_multiple('❌Invalid total amount,total amount should be ',str(calculated_total) +",but input is: " + self.invoice['Total'],color='red')
        else:
            print_color_multiple("✅Correct calculated total: ",round(calculated_total,2),color="green")
            
        # Calculate the Mall dollars and delivery fee
        if calculated_total >= 1000:
            mall_dollars = calculated_total // 10
        else:
            mall_dollars = 0

        # Check if the calculated Mall dollars match the provided Mall dollars
        if mall_dollars != mall_dollars:
            print_color_multiple('❌Invalid mall_dollars,mall_dollars should be ',str(mall_dollars) +",but input is: " + mall_dollars,color='red')
        else:
            print_color_multiple("✅Correct mall dollars: ",round(mall_dollars,1),color="green")
            
        print_color ("Invoice verified finished.",'blue')

In [130]:

print_color("Student Name: Yu Fo Hon","blue")
print_color("Student ID: 20201702","blue")
# Sample invoice data
invoice_data = {
    'order number': "A123456B5678784(7)",
    'number of items': 4,
    'items':['Item 1, 1121, Quantity, 4, price, 108.00','Item 2, 2134, Quantity, 4, price, 198.00','Item 3, 3019, Quantity, 3, price, 158.70','Item 4, 4018, Quantity, 1, price, 280.00'],
    'sub_total': 744.70,
    'VIP discount': -2.00,
    'VIPXX_discount': -37.14,  # Percentage discount after VIP discount
    'VIPXX_str':'VIP95 discount',
    'Total': 705.56,
    'Mall dollars': 0.0,
    'Delivery Fee':0,
}

file_path = "C:\\Users\\user\\Desktop\\invoice\\AI_ToneInvoice_TestData.txt"
invoice_data = InvoiceVerifier.from_file(file_path)
# Create an instance of InvoiceVerifier

for invoice in invoice_data:
    print_color_multiple("Start of test for invoice:", invoice, color='blue')
    invoice_verifier = InvoiceVerifier(invoice)
    invoice_verifier.verify_invoice()
    print_color("End of test for invoice:", 'blue')
    print_color("-------------------------------", 'blue')


Student Name: Yu Fo Hon
Student ID: 20201702
Start of test for invoice: {'items': ['Item 1, 1121, Quantity, 4, price, 108.00', 'Item 2, 2134, Quantity, 4, price, 198.00', 'Item 3, 3019, Quantity, 3, price, 158.70', 'Item 4, 4018, Quantity, 1, price, 280.00'], 'order number': 'A123456B5678784(7)', 'number of items': '4', 'sub_total': '744.70', 'VIP discount': '-2.00', 'VIPXX_str': 'VIP95 discount', 'VIPXX_discount': '-37.14', 'Delivery Fee': '0', 'Total': '705.56', 'Mall dollars': '0.0'}
Order number length:
✅
Modulus character:
✅
Digits in specified parts:
✅
Item count format:
✅
Check digit format:
✅
Parentheses format:
✅
First character:
✅
✅Correct check_digit:  7
✅Correct subtotal:  744.7
✅Correct vip discount:  -2.0
✅Correct VIP95 discount -37.14
✅Correct delivery fee:  0
✅Correct calculated total:  705.56
✅Correct mall dollars:  0
Invoice verified finished.
End of test for invoice:
-------------------------------
Start of test for invoice: {'items': ['Item 1, 1121, Quantity, 4, pri

In [131]:

invoice_data = InvoiceVerifier.from_file(file_path)
print(len(invoice_data))
print(invoice_data[2])

5
{'items': ['Item 1, 1234, Quantity, 3, price, 108.00', 'Item 2, 2345, Quantity, 4, price, 198.00', 'Item 3, 3456, Quantity, 3, price, 158.70', 'Item 4, 4567, Quantity, 1, price, 280.00', 'Item 5, 5678, Quantity, 3, price, 158.70', 'Item 6, 6789, Quantity, 1, price, 400.00'], 'order number': 'A123456B5678806(3)', 'number of items': '6', 'sub_total': '1303.40', 'VIP discount': '-0.00', 'VIPXX_str': 'VIP85 discount', 'VIPXX_discount': '-195.51', 'Delivery Fee': '0', 'Total': '1107.89', 'Mall dollars': '2.22'}
